Imports and global settings

In [1]:
import matplotlib.pyplot as plt
import mediapy as media
import mujoco
import numpy as np

from pathlib import Path
from tqdm import tqdm


res = (480, 640)
fps = 30
duration = 10.0

np.random.seed(420)

Model loading and renderer setup

In [2]:
model_dir = Path("scene")
model_xml = model_dir / "scene.xml"

# Load model.
model = mujoco.MjModel.from_xml_path(str(model_xml))
data = mujoco.MjData(model)

# Make sure offscreen rendering can support the desired resolution.
h, w = res
model.vis.global_.offheight = h
model.vis.global_.offwidth = w

renderer = mujoco.Renderer(model, height=h, width=w)     

# Rendering options for visual and collision geoms.
vis = mujoco.MjvOption()
vis.geomgroup[2] = True
vis.geomgroup[3] = False

coll = mujoco.MjvOption()
coll.geomgroup[2] = False
coll.geomgroup[3] = True
coll.flags[mujoco.mjtVisFlag.mjVIS_CONVEXHULL] = True

# Create a camera
camera = mujoco.MjvCamera()
mujoco.mjv_defaultFreeCamera(model, camera)
camera.distance = 2
offset = model.vis.global_.azimuth

Kernelized Movement Primitives

In [3]:
# Pull the KMP trajectories

kmp_model_dir = Path('trained_models')
pos_model = kmp_model_dir / 'sinusoidal_traj.npy'

pos = np.load(pos_model)

Controller settings

In [4]:

KP_pos = np.eye(3)*50
KD_pos = np.eye(3)*0.001
KP_rot = np.eye(3)*5
KD_rot = np.eye(3)*0.001

def to_axis_angle(quaternion):
    axis_angle = np.zeros((3, 1)) 
    # Extract the angle of rotation
    angle = 2*np.arccos(quaternion[0])  # Angle in radians
    # Avoid division by zero for small angles
    if np.abs(angle) > 1e-8:
        # Extract the axis of rotation
        axis = quaternion[1:] / np.sin(angle/2)
        axis_angle = (angle)*axis
    return axis_angle

def controller(model, data):
    """
    This function implements a P controller for tracking
    the reference motion.
    """
    # End-effector position
    end_eff_pos = data.sensordata[:3]
    
    # Compute end-effector Jacobians
    jacp = np.zeros((3,6))
    jacr = np.zeros((3,6))
    mujoco.mj_jac(model, data, jacp, jacr, end_eff_pos, 8)
    
    # Δq = Jinv * Δx
    J = np.vstack((jacp, jacr))
    KP = np.block([[KP_pos, np.zeros((3,3))],[np.zeros((3,3)), KP_rot]])
    KD = np.block([[KD_pos, np.zeros((3,3))],[np.zeros((3,3)), KD_rot]])
    actual_pos = np.concatenate((data.sensordata[:3], to_axis_angle(data.sensordata[3:7])))
    target_pos = np.concatenate((target, to_axis_angle(np.array([0,0,0,1]))))
    actual_speed = data.sensordata[7:]
    dx = KP @ (target_pos - actual_pos) 
    dx = KD @ (dx - actual_speed)
    global position
    position += dx * model.opt.timestep
    dq = np.linalg.pinv(J) @ position


    # Target position is q + Δq
    data.ctrl = data.qpos + dq

Simulation

In [5]:
# Reset the scene
mujoco.mj_resetData(model, data)

# First frame: home
home = np.array([0.0, -np.pi/2, 0.0, -np.pi/2, 0.0, 0.0])
data.qpos[:] = home
data.ctrl[:] = home

global position
position = np.concatenate((data.sensordata[:3],to_axis_angle(data.sensordata[3:7])))

# Set the controller
mujoco.set_mjcb_control(controller)

frames = []
nsteps = int(np.ceil(duration / model.opt.timestep))
for i in tqdm(range(nsteps)):
  j = i * pos.shape[-1] // nsteps
  target = np.array([0.5,0.5,pos[j]])
  mujoco.mj_step(model, data)
  if len(frames) < data.time * fps:
    renderer.update_scene(data, camera, scene_option=vis)
    frame = renderer.render().copy().astype(np.uint8)
    frames.append(frame)
media.show_video(frames, fps=fps, loop=False)

  0%|          | 0/10000 [00:00<?, ?it/s]

100%|██████████| 10000/10000 [00:10<00:00, 940.31it/s]


This browser does not support the video tag.
